In [1]:
#팩트 테이블(fact_sale), 차원 테이블(dimension_date, dimension_city)에 대한 참조를 생성

df_fact_sale = spark.read.table("gold_lakehouse.fact_sale") 
df_dimension_date = spark.read.table("gold_lakehouse.dimension_date")
df_dimension_city = spark.read.table("gold_lakehouse.dimension_city")

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 3, Finished, Available, Finished)

In [4]:
#집계 테이블(aggregate_sale_by_date_city) 생성

sale_by_date_city = df_fact_sale.alias("sale") \
.join(df_dimension_date.alias("date"), df_fact_sale.InvoiceDateKey == df_dimension_date.Date, "inner") \
.join(df_dimension_city.alias("city"), df_fact_sale.CityKey == df_dimension_city.CityKey, "inner") \
.select("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory", "sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.groupBy("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory")\
.sum("sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.withColumnRenamed("sum(TotalExcludingTax)", "SumOfTotalExcludingTax")\
.withColumnRenamed("sum(TaxAmount)", "SumOfTaxAmount")\
.withColumnRenamed("sum(TotalIncludingTax)", "SumOfTotalIncludingTax")\
.withColumnRenamed("sum(Profit)", "SumOfProfit")\
.orderBy("date.Date", "city.StateProvince", "city.City")

sale_by_date_city.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/aggregate_sale_by_date_city")

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 6, Finished, Available, Finished)

In [10]:
%%sql
SELECT * 
FROM aggregate_sale_by_date_city;

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 12, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 12 fields>

In [11]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW sale_by_date_employee
AS
SELECT
	DD.Date, DD.CalendarMonthLabel
    , DD.Day, DD.ShortMonth Month, CalendarYear Year
	,DE.PreferredName, DE.Employee
	,SUM(FS.TotalExcludingTax) SumOfTotalExcludingTax
	,SUM(FS.TaxAmount) SumOfTaxAmount
	,SUM(FS.TotalIncludingTax) SumOfTotalIncludingTax
	,SUM(Profit) SumOfProfit 
FROM gold_lakehouse.fact_sale FS
INNER JOIN gold_lakehouse.dimension_date DD ON FS.InvoiceDateKey = DD.Date
INNER JOIN gold_lakehouse.dimension_Employee DE ON FS.SalespersonKey = DE.EmployeeKey
GROUP BY DD.Date, DD.CalendarMonthLabel, DD.Day, DD.ShortMonth, DD.CalendarYear, DE.PreferredName, DE.Employee
ORDER BY DD.Date ASC, DE.PreferredName ASC, DE.Employee ASC

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
sale_by_date_employee = spark.sql("SELECT * FROM sale_by_date_employee")
sale_by_date_employee.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/aggregate_sale_by_date_employee")

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 14, Finished, Available, Finished)

In [13]:
%%sql
SELECT *
FROM aggregate_sale_by_date_employee;

StatementMeta(, 80ae4e23-fa92-4afe-b15a-3794790c378b, 15, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 11 fields>